# Install Requirements

In [5]:
# 1. 彻底清理旧环境
# %pip uninstall -y torch torchvision torchaudio onnx onnxruntime onnxruntime-gpu

# 2. 安装适配 RTX 50 (Blackwell) 的 PyTorch Nightly
%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

# 3. 安装最新版 ONNX 工具链
%pip install git+https://github.com/openai/CLIP.git
%pip install onnx>=1.16.0 onnxconverter-common>=1.13.0 onnxruntime-gpu>=1.18.0
%pip install onnxscript
%pip install onnxruntime
%pip install tqdm
%pip install scipy
%pip install pycocotools

import torch
print(f"🔥 PyTorch Version: {torch.__version__}")
print(f"🚀 CUDA Available: {torch.cuda.is_available()}")

Error processing line 1 of /home/guiyan/miniforge3/envs/clip/lib/python3.11/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 195, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Looking in indexes: https://download.pytorch.org/whl/nightly/cu128

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Error processing line 1 of /home/guiyan/miniforge3/envs/clip/lib/python3.11/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 195, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9_t8qyae
  Running command gi

## Export Image Tower to ONNX

In [3]:
import clip
from PIL import Image
from torch import Tensor
import torch
import torch.onnx
import os
from pathlib import Path

# 1. 强制使用 CPU 进行导出 (为了最大化 Android 兼容性)
device = "cpu"

print("正在加载 CLIP 模型...")
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
vit = model.visual
vit.eval()

image_path = Path("image.jpeg")

if not image_path.exists():
    raise FileNotFoundError(f"找不到图片，请检查路径: {image_path.absolute()}")

print(f"正在读取图片: {image_path}")
i = Image.open(image_path)

input_tensor: Tensor = preprocess(i).unsqueeze(0).to(device)
output_dir = Path("../../model")
output_dir.mkdir(parents=True, exist_ok=True)
model_output_path = output_dir / "clip-image-encoder.onnx"
print(f"🚀 开始导出 Image Encoder")
dynamic_shapes = {
    "x": {0: torch.export.Dim("batch", min=1, max=32)}
}
torch.onnx.export(
    vit,
    (input_tensor,),
    str(model_output_path),
    export_params=True,
    opset_version=18,
    do_constant_folding=True,
    dynamic_shapes=dynamic_shapes,
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    input_names=['input'],
    output_names=['output']
)

print(f"✅ 导出成功！文件保存在当前目录: {model_output_path}")

正在加载 CLIP 模型...
正在读取图片: image.jpeg
🚀 开始导出 Image Encoder
[torch.onnx] Obtain model graph for `VisionTransformer([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `VisionTransformer([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decompositions...


/home/guiyan/miniforge3/envs/clip/lib/python3.11/copyreg.py:105: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decompositions... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
[torch.onnx] Optimize the ONNX graph...
Applied 13 of general pattern rewrite rules.
[torch.onnx] Optimize the ONNX graph... ✅
✅ 导出成功！文件保存在当前目录: ../../model/clip-image-encoder.onnx


## Quantization Image Model

In [3]:
import os
from pathlib import Path
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process

model_dir = Path("../../model")
model_image = model_dir / "clip-image-encoder.onnx"
model_image_prep = model_dir / "clip-image-encoder-quant-pre.onnx"
model_image_quant = model_dir / "clip-image-encoder-quant-int8.onnx"


if not model_image.exists():
    raise FileNotFoundError(f"找不到原始模型: {model_image.absolute()}")


print("🔨 正在进行量化预处理...")
quant_pre_process(
    input_model=str(model_image),
    output_model_path=str(model_image_prep)
)


print("📉 正在进行 INT8 动态量化...")

quantize_dynamic(
    model_input=str(model_image_prep),
    model_output=str(model_image_quant),
    weight_type=QuantType.QInt8,
    nodes_to_exclude=['/conv1/Conv']
)

print(f"✅ 量化完成！")

original_size = (model_image.stat().st_size + Path(str(model_image)+".data").stat().st_size) / (1024 * 1024)
quantized_size = model_image_quant.stat().st_size / (1024 * 1024)

print(f"📊 原始总体积 (含.data): {original_size:.2f} MB")
print(f"📊 量化后体积: {quantized_size:.2f} MB")
print(f"📉 压缩比: {(1 - quantized_size/original_size)*100:.1f}%")

🔨 正在进行量化预处理...
📉 正在进行 INT8 动态量化...
✅ 量化完成！
📊 原始总体积 (含.data): 336.02 MB
📊 量化后体积: 84.37 MB
📉 压缩比: 74.9%


## Benchmark Image Model

In [9]:
import torch
import onnxruntime as ort
import numpy as np
from torchvision.datasets import Flowers102
from torchvision import transforms
from tqdm import tqdm
import time
from pathlib import Path

# 1. 基础配置
MODEL_DIR = Path("../../model")
FP32_PATH = MODEL_DIR / "clip-image-encoder.onnx"
INT8_PATH = MODEL_DIR / "clip-image-encoder-quant-int8.onnx"
DATA_ROOT = Path("../../data/dataset")
LIMIT = 1000

# 2. 预处理计时 (模拟 Android 端单张处理逻辑)
print(f"📦 正在测试预处理耗时 ({LIMIT} 张图片)...")
dataset = Flowers102(root=str(DATA_ROOT), split="test", download=True)
preprocess = transforms.Compose([
    transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
])

preprocessed_tensors = []
start_pre = time.perf_counter()

for i in tqdm(range(LIMIT), desc="Preprocessing"):
    img, _ = dataset[i]
    tensor = preprocess(img).unsqueeze(0).numpy()
    preprocessed_tensors.append(tensor)

end_pre = time.perf_counter()
lat_pre = (end_pre - start_pre) / LIMIT * 1000  # 平均每张预处理耗时 (ms)

# 3. 初始化会话
opts = ort.SessionOptions()
opts.intra_op_num_threads = 8
sess_fp32 = ort.InferenceSession(str(FP32_PATH), opts, providers=['CPUExecutionProvider'])
sess_int8 = ort.InferenceSession(str(INT8_PATH), opts, providers=['CPUExecutionProvider'])

# 4. 核心推理对比测试函数
def run_pure_inference(session, tensors, name):
    input_name = session.get_inputs()[0].name
    # 预热
    for i in range(10):
        session.run(None, {input_name: tensors[i]})

    start_time = time.perf_counter()
    outputs = []
    for tensor in tqdm(tensors, desc=f"Inference ({name})"):
        out = session.run(None, {input_name: tensor})[0]
        outputs.append(out)
    end_time = time.perf_counter()

    avg_latency = (end_time - start_time) / len(tensors) * 1000
    return avg_latency, np.concatenate(outputs)

# 运行推理测试
lat_fp32, feats_fp32 = run_pure_inference(sess_fp32, preprocessed_tensors, "FP32")
lat_int8, feats_int8 = run_pure_inference(sess_int8, preprocessed_tensors, "INT8")

# 5. 计算指标
speedup_pure = lat_fp32 / lat_int8
total_lat_int8 = lat_pre + lat_int8  # 总 Pipeline 耗时

# 精度与存储
sims = torch.nn.functional.cosine_similarity(torch.from_numpy(feats_fp32), torch.from_numpy(feats_int8))
avg_sim = sims.mean().item()
size_fp32 = (FP32_PATH.stat().st_size + Path(str(FP32_PATH)+".data").stat().st_size) / (1024*1024)
size_int8 = INT8_PATH.stat().st_size / (1024*1024)

# 6. 打印报告
print("\n" + "="*50)
print("📊 CLIP Image Encoder 全流程性能报告")
print("="*50)
print(f"【单张图片耗时分解】")
print(f" - 预处理 (Resize/Normalize): {lat_pre:.2f} ms")
print(f" - 纯推理 (INT8 Inference): {lat_int8:.2f} ms")
print(f" - 🏁 总 Pipeline (INT8): {total_lat_int8:.2f} ms")

print(f"\n【核心加速比 (INT8 vs FP32)】")
print(f" - 纯推理加速: {speedup_pure:.2f}x")
print(f" - 全流程加速: {(lat_pre + lat_fp32) / total_lat_int8:.2f}x")

print(f"\n【量化精度与存储】")
print(f" - 平均向量相似度: {avg_sim:.6f}")
print(f" - 存储空间对比: {size_fp32:.2f} MB -> {size_int8:.2f} MB")

print(f"\n【5 万张图扫描总耗时预估 (INT8)】")
# 公式：(预处理时间 + 推理时间) * 50000 / 1000 / 60
total_min = (50000 * total_lat_int8) / 1000 / 60
print(f" - 预计扫描总用时: {total_min:.2f} 分钟")
print(f" - 其中预处理占总比: {(lat_pre / total_lat_int8) * 100:.1f}%")
print("="*50)

📦 正在测试预处理耗时 (1000 张图片)...


Inference (INT8): 100%|██████████| 1000/1000 [00:11<00:00, 85.06it/s]


📊 CLIP Image Encoder 全流程性能报告
【单张图片耗时分解】
 - 预处理 (Resize/Normalize): 4.38 ms
 - 纯推理 (INT8 Inference): 11.76 ms
 - 🏁 总 Pipeline (INT8): 16.14 ms

【核心加速比 (INT8 vs FP32)】
 - 纯推理加速: 1.80x
 - 全流程加速: 1.58x

【量化精度与存储】
 - 平均向量相似度: 0.969175
 - 存储空间对比: 336.02 MB -> 84.37 MB

【5 万张图扫描总耗时预估 (INT8)】
 - 预计扫描总用时: 13.45 分钟
 - 其中预处理占总比: 27.1%


---

# Text Encoder

## TextEncoder Structure

copied from Openai.CLIP

In [10]:
import torch.nn as nn
from collections import OrderedDict

import torch
import numpy as np
import clip

class ResidualAttentionBlock(nn.Module):
    def __init__(self, d_model: int, n_head: int, attn_mask: torch.Tensor = None):
        super().__init__()

        self.attn = nn.MultiheadAttention(d_model, n_head)
        self.ln_1 = LayerNorm(d_model)
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc", nn.Linear(d_model, d_model * 4)),
            ("gelu", QuickGELU()),
            ("c_proj", nn.Linear(d_model * 4, d_model))
        ]))
        self.ln_2 = LayerNorm(d_model)
        self.attn_mask = attn_mask

    def attention(self, x: torch.Tensor):
        self.attn_mask = self.attn_mask.to(dtype=x.dtype, device=x.device) if self.attn_mask is not None else None
        return self.attn(x, x, x, need_weights=False, attn_mask=self.attn_mask)[0]

    def forward(self, x: torch.Tensor):
        x = x + self.attention(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class Transformer(nn.Module):
    def __init__(self, width: int, layers: int, heads: int, attn_mask: torch.Tensor = None):
        super().__init__()
        self.width = width
        self.layers = layers
        self.resblocks = nn.Sequential(*[ResidualAttentionBlock(width, heads, attn_mask) for _ in range(layers)])

    def forward(self, x: torch.Tensor):
        return self.resblocks(x)


class LayerNorm(nn.LayerNorm):
    """Subclass torch's LayerNorm to handle fp16."""

    def forward(self, x: torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)


class QuickGELU(nn.Module):
    def forward(self, x: torch.Tensor):
        return x * torch.sigmoid(1.702 * x)


class TextEncoder(nn.Module):
    def __init__(self,
          embed_dim: int,
          # text
          context_length: int,
          vocab_size: int,
          transformer_width: int,
          transformer_heads: int,
          transformer_layers: int
          ):
        super().__init__()

        self.context_length = context_length

        self.transformer = Transformer(
            width=transformer_width,
            layers=transformer_layers,
            heads=transformer_heads,
            attn_mask=self.build_attention_mask()
        )

        self.vocab_size = vocab_size
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)
        self.positional_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))
        self.ln_final = LayerNorm(transformer_width)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        self.temperature = nn.Parameter(torch.tensor(0.07))

        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))

        print(f"text_projection shape: {self.text_projection.shape}")
        self.dtype = torch.float32

        self.initialize_parameters()

    def initialize_parameters(self):
        nn.init.normal_(self.token_embedding.weight, std=0.02)
        nn.init.normal_(self.positional_embedding, std=0.01)

        proj_std = (self.transformer.width ** -0.5) * ((2 * self.transformer.layers) ** -0.5)
        attn_std = self.transformer.width ** -0.5
        fc_std = (2 * self.transformer.width) ** -0.5
        for block in self.transformer.resblocks:
            nn.init.normal_(block.attn.in_proj_weight, std=attn_std)
            nn.init.normal_(block.attn.out_proj.weight, std=proj_std)
            nn.init.normal_(block.mlp.c_fc.weight, std=fc_std)
            nn.init.normal_(block.mlp.c_proj.weight, std=proj_std)

        if self.text_projection is not None:
            nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)
        else:
            nn.init.normal_(self.text_projection, std=self.custom_text_config['text_rep_size'] ** -0.5)

    def build_attention_mask(self):
        # lazily create causal attention mask, with full attention between the vision tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = torch.empty(self.context_length, self.context_length)
        mask.fill_(float("-inf"))
        mask.triu_(1)  # zero out the lower diagonal
        return mask

    def forward(self, text):
        # print(f'text: {text}')
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)
        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection

        return x


## Export Text Tower to ONNX

In [22]:
import torch
import clip
from pathlib import Path

# 1. 强制使用 CPU
device = "cpu"

print("正在加载 CLIP 模型...")
official_model, _ = clip.load("ViT-B/32", device=device)

# 2. 实例化自定义 TextEncoder
text_encoder = TextEncoder(
    embed_dim=512,
    context_length=77,
    vocab_size=49408,
    transformer_width=512,
    transformer_heads=8,
    transformer_layers=12
).to(device)

# 3. 灌装权重
text_encoder.load_state_dict(official_model.state_dict(), strict=False)
text_encoder.eval()

# 4. 路径设置
output_dir = Path("../../model")
output_dir.mkdir(parents=True, exist_ok=True)
model_text_path = output_dir / "clip-text-encoder.onnx"

# 5. 准备模拟输入：模拟输入 2 条文本，增加导出时的鲁棒性
dummy_input = torch.ones(2, 77).long().to(device)

print(f"🚀 开始导出 Text Encoder")

dynamic_shapes = {
    "text": {0: torch.export.Dim("batch", min=1, max=32)}
}

torch.onnx.export(
    text_encoder,
    (dummy_input,),
    str(model_text_path),
    export_params=True,
    opset_version=18,
    do_constant_folding=True,
    dynamic_shapes=dynamic_shapes,
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    },
    input_names=['input'],
    output_names=['output']
)

print(f"✅ 导出成功！文件保存在: {model_text_path}")

正在加载 CLIP 模型...
text_projection shape: torch.Size([512, 512])
🚀 开始导出 Text Encoder
[torch.onnx] Obtain model graph for `TextEncoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `TextEncoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decompositions...


/home/guiyan/miniforge3/envs/clip/lib/python3.11/copyreg.py:105: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
W0217 22:47:54.958000 118430 site-packages/torch/onnx/_internal/exporter/_core.py:1253] Tensor 'transformer.resblocks.1.attn_mask' is not one of the initializers
W0217 22:47:54.958000 118430 site-packages/torch/onnx/_internal/exporter/_core.py:1253] Tensor 'transformer.resblocks.2.attn_mask' is not one of the initializers
W0217 22:47:54.958000 118430 site-packages/torch/onnx/_internal/exporter/_core.py:1253] Tensor 'transformer.resblocks.3.attn_mask' is not one of the initializers
W0217 22:47:54.959000 118430 site-packages/torch/onnx/_internal/exporter/_core.py:1253] Tensor 'transformer.resblocks.4.attn_mask' is not one of the initializers
W0217 22:47:54.959000 118430 site-packages/torch/onnx/_internal/exporter/_core.py:1253] Tensor 'transformer.resblocks.5.at

[torch.onnx] Run decompositions... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
[torch.onnx] Optimize the ONNX graph...
Applied 62 of general pattern rewrite rules.
[torch.onnx] Optimize the ONNX graph... ✅
✅ 导出成功！文件保存在: ../../model/clip-text-encoder.onnx


## Quantization Text Model

In [21]:
import os
from pathlib import Path
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process

# 1. 路径设置 (对齐项目结构)
model_dir = Path("../../model")
model_text = model_dir / "clip-text-encoder.onnx"
model_text_prep = model_dir / "clip-text-encoder-quant-pre.onnx"
model_text_quant = model_dir / "clip-text-encoder-quant-int8.onnx"

# 检查原始文件是否存在
if not model_text.exists():
    raise FileNotFoundError(f"找不到原始文本模型: {model_text.absolute()}")

# 2. 量化预处理 (Pre-process)
# 这一步会优化计算图，确保 Transformer 的注意力机制在量化后依然准确
print("🔨 正在进行文本模型量化预处理...")
quant_pre_process(
    input_model=str(model_text),
    output_model_path=str(model_text_prep),
    skip_symbolic_shape=True
)

# 3. 执行动态量化 (INT8)
print("📉 正在生成 INT8 文本模型...")
quantize_dynamic(
    model_input=str(model_text_prep),
    model_output=str(model_text_quant),
    weight_type=QuantType.QInt8,
)

fp32_onnx = model_dir / "clip-text-encoder.onnx"
fp32_data = model_dir / "clip-text-encoder.onnx.data"

size_fp32_total = fp32_onnx.stat().st_size
if fp32_data.exists():
    size_fp32_total += fp32_data.stat().st_size

size_orig = size_fp32_total / (1024 * 1024)
size_quant = model_text_quant.stat().st_size / (1024 * 1024)

print("\n" + "="*40)
print(f"✅ 文本模型量化完成！")
print(f"📊 原始体积: {size_orig:.2f} MB")
print(f"📊 量化体积: {size_quant:.2f} MB")
print(f"📉 压缩比: {(1 - size_quant/size_orig)*100:.1f}%")
print("="*40)

🔨 正在进行文本模型量化预处理...
📉 正在生成 INT8 文本模型...

✅ 文本模型量化完成！
📊 原始体积: 243.27 MB
📊 量化体积: 62.07 MB
📉 压缩比: 74.5%


## Benchmark Text Model

In [20]:
import os
import sys
import zipfile
import urllib.request
import json
import time
from pathlib import Path

import torch
import clip
import onnxruntime as ort
import numpy as np
from tqdm import tqdm

# 1. 基础配置
MODEL_DIR = Path("../../model")
DATA_ROOT = Path("../../data/dataset/coco")
FP32_PATH = MODEL_DIR / "clip-text-encoder.onnx"
INT8_PATH = MODEL_DIR / "clip-text-encoder-quant-int8.onnx"

# COCO 2017 标注文件下载地址
COCO_ANN_URL = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
COCO_ANN_JSON = DATA_ROOT / "annotations/captions_val2017.json"
LIMIT = 1000

# 2. 自动化数据管理逻辑
def ensure_coco_annotations():
    """检查并自动下载 COCO 标注文件"""
    if COCO_ANN_JSON.exists():
        print(f"✅ 找到 COCO 标注文件: {COCO_ANN_JSON}")
        return

    print(f"📦 找不到标注文件，准备从官网拉取 (约 241MB)...")
    DATA_ROOT.mkdir(parents=True, exist_ok=True)
    zip_path = DATA_ROOT / "annotations_trainval2017.zip"

    # 下载进度条
    def report_progress(block_num, block_size, total_size):
        read_so_far = block_num * block_size
        if total_size > 0:
            percent = read_so_far * 1e2 / total_size
            s = f"\r正在下载: {percent:5.1f}% [{read_so_far / 1e6:.1f}MB / {total_size / 1e6:.1f}MB]"
            sys.stdout.write(s)
            sys.stdout.flush()

    try:
        urllib.request.urlretrieve(COCO_ANN_URL, zip_path, reporthook=report_progress)
        print("\n\n解压中...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DATA_ROOT)
        zip_path.unlink()  # 删除压缩包节省空间
        print("✨ 数据准备就绪！")
    except Exception as e:
        print(f"\n❌ 下载失败: {e}")
        print("请检查网络链接或尝试手动下载放置。")
        sys.exit(1)

# 执行数据检查
ensure_coco_annotations()

# 3. 加载真实语料 (MS COCO)
print(f"📦 正在从 MS COCO 加载真实人类标注...")
try:
    with open(COCO_ANN_JSON, 'r') as f:
        data = json.load(f)
    # 提取前 LIMIT 条文本描述
    search_queries = [ann['caption'] for ann in data['annotations'][:LIMIT]]
    print(f"✅ 成功加载 {len(search_queries)} 条 COCO 真实描述。")
except Exception as e:
    print(f"⚠️ 读取 JSON 失败: {e}")
    sys.exit(1)

# 4. 预处理计时 (BPE Tokenization)
print(f"⏱️ 正在测试 Tokenization 耗时...")
start_pre = time.perf_counter()

tokenized_list = []
for q in search_queries:
    # 模拟 Android 端的逻辑：裁剪或填充至 77 位
    tokens = clip.tokenize([q]).numpy().astype(np.int64)
    tokenized_list.append(tokens)

end_pre = time.perf_counter()
lat_pre = (end_pre - start_pre) / LIMIT * 1000  # 平均单条耗时 (ms)

# 5. 初始化推理会话
opts = ort.SessionOptions()
opts.intra_op_num_threads = 8
sess_fp32 = ort.InferenceSession(str(FP32_PATH), opts, providers=['CPUExecutionProvider'])
sess_int8 = ort.InferenceSession(str(INT8_PATH), opts, providers=['CPUExecutionProvider'])

# 6. 核心推理对比
def run_text_benchmark(session, tokens_list, name):
    input_name = session.get_inputs()[0].name
    # 预热
    for i in range(10):
        session.run(None, {input_name: tokens_list[i]})

    latencies = []
    features = []

    for tokens in tqdm(tokens_list, desc=f"Inference ({name})"):
        t0 = time.perf_counter()
        out = session.run(None, {input_name: tokens})[0]
        latencies.append(time.perf_counter() - t0)
        features.append(out)

    avg_latency = np.mean(latencies) * 1000
    return avg_latency, np.concatenate(features)

lat_fp32, feats_fp32 = run_text_benchmark(sess_fp32, tokenized_list, "FP32")
lat_int8, feats_int8 = run_text_benchmark(sess_int8, tokenized_list, "INT8")

# 7. 计算指标
speedup_pure = lat_fp32 / lat_int8
cos_sim = torch.nn.functional.cosine_similarity(
    torch.from_numpy(feats_fp32),
    torch.from_numpy(feats_int8)
).mean().item()

# 存储空间
def get_model_size(p):
    size = p.stat().st_size
    data_p = Path(str(p) + ".data")
    if data_p.exists():
        size += data_p.stat().st_size
    return size / (1024 * 1024)

size_fp32 = get_model_size(FP32_PATH)
size_int8 = get_model_size(INT8_PATH)

# 8. 打印报告
print("\n" + "="*50)
print("📊 CLIP Text Encoder 实战压力测试报告 (MS COCO 自动化版)")
print("="*50)
print(f"【推理性能分析】")
print(f" - BPE 分词耗时: {lat_pre:.4f} ms")
print(f" - FP32 平均耗时: {lat_fp32:.2f} ms")
print(f" - INT8 平均耗时: {lat_int8:.2f} ms")
print(f" - 🚀 核心加速比: {speedup_pure:.2f}x")

print(f"\n【量化质量监控】")
print(f" - 向量平均相似度: {cos_sim:.6f}")
print(f" - 🎯 语义回退 (Precision Loss): {(1 - cos_sim)*100:.4f}%")

print(f"\n【资源占用对比】")
print(f" - 磁盘/内存占用: {size_fp32:.2f} MB -> {size_int8:.2f} MB")
print(f" - 📉 节省空间: {size_fp32 - size_int8:.2f} MB")

print(f"\n【Android 端实战体感】")
total_int8 = lat_pre + lat_int8
print(f" - 用户搜索预期延迟: ~{total_int8:.2f} ms")
print(f" - 体验评级: {'⚡ 极速 (无感知)' if total_int8 < 50 else '✅ 流畅'}")
print("="*50)

✅ 找到 COCO 标注文件: ../../data/dataset/coco/annotations/captions_val2017.json
📦 正在从 MS COCO 加载真实人类标注...
✅ 成功加载 1000 条 COCO 真实描述。
⏱️ 正在测试 Tokenization 耗时...


Inference (INT8): 100%|██████████| 1000/1000 [00:07<00:00, 125.39it/s]


📊 CLIP Text Encoder 实战压力测试报告 (MS COCO 自动化版)
【推理性能分析】
 - BPE 分词耗时: 0.0670 ms
 - FP32 平均耗时: 13.14 ms
 - INT8 平均耗时: 7.93 ms
 - 🚀 核心加速比: 1.66x

【量化质量监控】
 - 向量平均相似度: 0.830977
 - 🎯 语义回退 (Precision Loss): 16.9023%

【资源占用对比】
 - 磁盘/内存占用: 243.27 MB -> 62.07 MB
 - 📉 节省空间: 181.20 MB

【Android 端实战体感】
 - 用户搜索预期延迟: ~8.00 ms
 - 体验评级: ⚡ 极速 (无感知)


# Export to ORT format

In [29]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort ../../model/clip-image-encoder-quant-int8.onnx

Error processing line 1 of /home/guiyan/miniforge3/envs/clip/lib/python3.11/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 195, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/guiyan/project/CLIP-model/model/clip-image-encoder-quant-int8.onnx to ORT format model /home/guiyan/project/CLIP-model/model/clip-image-encoder-quant-int8.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2026-02-17 22:55:04,814 ort_format_model.utils [INFO] - Created config in /home/guiyan/project/CLIP-model/model/clip-image-encoder-quant-int8.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /home/guiya

In [30]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort ../../model/clip-text-encoder-quant-int8.onnx

Error processing line 1 of /home/guiyan/miniforge3/envs/clip/lib/python3.11/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 195, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/guiyan/project/CLIP-model/model/clip-text-encoder-quant-int8.onnx to ORT format model /home/guiyan/project/CLIP-model/model/clip-text-encoder-quant-int8.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2026-02-17 22:55:10,699 ort_format_model.utils [INFO] - Created config in /home/guiyan/project/CLIP-model/model/clip-text-encoder-quant-int8.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /home/guiyan/p